#Importing the libraries

In [3]:
import keras
import numpy as np 
import pandas as pd 
from hyperopt import STATUS_OK , Trials, fmin, hp , tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow 
from mlflow.models import  infer_signature 


In [9]:
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=';')

In [10]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [11]:
data.shape

(4898, 12)

Split the data into train , validation  and test

In [12]:
train , test = train_test_split(data, test_size=0.25 , random_state=42)

In [ ]:
train_x = train.drop(columns=["quality"], axis=1).values
train_y = train["quality"].values.ravel()

#test data
test_x = test.drop(columns=["quality"], axis=1)
test_y = test["quality"].values.ravel()
# train & validation data
train_x , valid_x , train_y , valid_y = train_test_split(train_x, train_y , test_size=0.20 , random_state=42)
